# Question revision chain

- get a question
- LLM A and B verify if the question is OK
    - there is more than one option correct
    - sugests modifications to improve the questions and options
    - sugest a detailed explanation to give to the student
- verify if 
    correct_answer is equal for both
    AND
    there is just one option true 
    - if YES: just take the explanations from gemini and set the correct answer
    - if NO: 
        - if there is more than one option
            - if YES:
                - if it is not passing this part for the second time
                    - if YES:
                        - fix the options
                        - go to the start of flow
                    - if NO: 
                        - discart this question because it already tried to fix it
            - if NO: 
                - take a vote from openai
                - go to the part of save the question because it should have a majority vote for one option

            


In [1]:
from dotenv import load_dotenv
import json_repair
import json
import os
import pandas as pd

from pydantic import BaseModel, Field
from typing import Literal

In [2]:
df = pd.read_csv("questions.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 953 non-null    int64 
 1   created_at         953 non-null    object
 2   subject_matter     953 non-null    object
 3   topic_description  953 non-null    object
 4   level              953 non-null    object
 5   question           953 non-null    object
 6   type               953 non-null    object
 7   answer_correct     953 non-null    object
 8   explanation        953 non-null    object
 9   answer_a           953 non-null    object
 10  answer_b           953 non-null    object
 11  answer_c           953 non-null    object
 12  answer_d           953 non-null    object
dtypes: int64(1), object(12)
memory usage: 96.9+ KB


In [3]:
question = df.iloc[0]
question

id                                                                  11
created_at                               2024-09-22 12:31:43.546285+00
subject_matter                                             Probability
topic_description                       Understanding Confusion Matrix
level                                                     1 - Remember
question             Which term represents correctly classified pos...
type                                                  multiple_options
answer_correct                                                       a
explanation          True Positives (TP) represent the number of ca...
answer_a                                                True Positives
answer_b                                               False Positives
answer_c                                               False Negatives
answer_d                                                True Negatives
Name: 0, dtype: object

In [4]:
question["question"]

'Which term represents correctly classified positive cases?'

## Gemini

In [5]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

key_gemini = os.environ["GOOGLE_API_KEY"]

genai.configure(api_key=key_gemini)

/workspaces/quiz_interview/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
schema = content.Schema(
    type=content.Type.OBJECT,
    properties={
        "option_a_explanation": content.Schema(
            type=content.Type.STRING,
            description="Detailed explanation of why the option 'a' is correct or not.",
        ),
        "option_b_explanation": content.Schema(
            type=content.Type.STRING,
            description="Detailed explanation of why the option 'b' is correct or not.",
        ),
        "option_c_explanation": content.Schema(
            type=content.Type.STRING,
            description="Detailed explanation of why the option 'c' is correct or not.",
        ),
        "option_d_explanation": content.Schema(
            type=content.Type.STRING,
            description="Detailed explanation of why the option 'd' is correct or not.",
        ),
        "just_one_option_correct": content.Schema(
            type=content.Type.BOOLEAN,
            description="If there is just one option correct.",
        ),
        "correct_answer": content.Schema(
            type=content.Type.STRING,
            description="The correct option. It could be 'a', 'b', 'c', 'd'.",
        ),
    },
    required=[
        "option_a_explanation",
        "option_b_explanation",
        "option_c_explanation",
        "option_d_explanation",
        "just_one_option_correct",
        "correct_answer",
    ]
)

In [7]:
# Create the model
generation_config = {
    "temperature": 0.5,
    "top_p": 0.95,
    # "top_k": 64,
    # "max_output_tokens": 8192,
    "response_schema": schema,
    "response_mime_type": "application/json",
}
    

llm_gemini = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
)

In [8]:
prompt_revision = (
    "# BACKGROUND INFORMATION OF YOUR ROLE:\n"
    "You are an expert in machine learning.\n"
    "You are verifying if a question from a bank of question is ok to be used in a job interview.\n"
    "your task is to analyse the provided question to verify if there is some problem.\n"
    "You shoud verify if there is just one option correct or if there is more than one.\n"
    "The question could be perfect fine, so you could say it too.\n"
    "Also, you should give a detailed explanation for your answer.\n"
    "# QUESTION\n\n"
    "{question}\n\n"
    "# Option a\n"
    "{option_a}\n\n"
    "# Option b\n"
    "{option_b}\n\n"
    "# Option c\n"
    "{option_c}\n\n"
    "# Option d\n"
    "{option_d}\n\n"
)

In [10]:
prompt_revision_fromatted = prompt_revision.format(
        question=question["question"],
        option_a=question["answer_a"],
        option_b=question["answer_b"],
        option_c=question["answer_c"],
        option_d=question["answer_d"],        
    )

print( prompt_revision_fromatted )

# BACKGROUND INFORMATION OF YOUR ROLE:
You are an expert in machine learning.
You are verifying if a question from a bank of question is ok to be used in a job interview.
your task is to analyse the provided question to verify if there is some problem.
You shoud verify if there is just one option correct or if there is more than one.
The question could be perfect fine, so you could say it too.
Also, you should give a detailed explanation for your answer.
# QUESTION

Which term represents correctly classified positive cases?

# Option a
True Positives

# Option b
False Positives

# Option c
False Negatives

# Option d
True Negatives




In [11]:
response = llm_gemini.generate_content(prompt_revision_fromatted)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\"correct_answer\": \"a\", \"just_one_option_correct\": true, \"option_a_explanation\": \"True Positives (TP) represent the cases where the model correctly predicts a positive outcome when the actual outcome is also positive. It is a measure of how well the model identifies positive cases.\", \"option_b_explanation\": \"False Positives (FP) represent the cases where the model incorrectly predicts a positive outcome when the actual outcome is negative. It is a measure of how often the model incorrectly identifies a positive case.\", \"option_c_explanation\": \"False Negatives (FN) represent the cases where the model incorrectly predicts a negative outcome when the actual outcome is positive. It is a measure of how often the model fails to identify a positive

In [12]:
gemeni_analysis = json_repair.loads(response.text)
gemeni_analysis

{'correct_answer': 'a',
 'just_one_option_correct': True,
 'option_a_explanation': 'True Positives (TP) represent the cases where the model correctly predicts a positive outcome when the actual outcome is also positive. It is a measure of how well the model identifies positive cases.',
 'option_b_explanation': 'False Positives (FP) represent the cases where the model incorrectly predicts a positive outcome when the actual outcome is negative. It is a measure of how often the model incorrectly identifies a positive case.',
 'option_c_explanation': 'False Negatives (FN) represent the cases where the model incorrectly predicts a negative outcome when the actual outcome is positive. It is a measure of how often the model fails to identify a positive case.',
 'option_d_explanation': 'True Negatives (TN) represent the cases where the model correctly predicts a negative outcome when the actual outcome is also negative. It is a measure of how well the model identifies negative cases.'}

## Llama 3.1

In [13]:
class ExplanationSchema(BaseModel):
    option_a_explanation: str = Field(..., description="Detailed explanation of why the option 'a' is correct or not.")
    option_b_explanation: str = Field(..., description="Detailed explanation of why the option 'b' is correct or not.")
    option_c_explanation: str = Field(..., description="Detailed explanation of why the option 'c' is correct or not.")
    option_d_explanation: str = Field(..., description="Detailed explanation of why the option 'd' is correct or not.")
    just_one_option_correct: bool = Field(..., description="If there is just one option correct.")
    correct_answer: Literal['a', 'b', 'c', 'd'] = Field(..., description="The correct option. It could be 'a', 'b', 'c', or 'd'.")


In [14]:
from groq import Groq

key_groq = os.environ["GROQ_API_KEY"]
client = Groq(api_key=key_groq)

In [15]:
prompt_revision_with_json = prompt_revision + (
    "# OUTPUT FORMAT: json\n\n"
    "{output_format}"
)

In [16]:
prompt_revision_fromatted = prompt_revision_with_json.format(
        question=question["question"],
        option_a=question["answer_a"],
        option_b=question["answer_b"],
        option_c=question["answer_c"],
        option_d=question["answer_d"],
        output_format=json.dumps(ExplanationSchema.model_json_schema(), indent=2)
    )

print( prompt_revision_fromatted )

# BACKGROUND INFORMATION OF YOUR ROLE:
You are an expert in machine learning.
You are verifying if a question from a bank of question is ok to be used in a job interview.
your task is to analyse the provided question to verify if there is some problem.
You shoud verify if there is just one option correct or if there is more than one.
The question could be perfect fine, so you could say it too.
Also, you should give a detailed explanation for your answer.
# QUESTION

Which term represents correctly classified positive cases?

# Option a
True Positives

# Option b
False Positives

# Option c
False Negatives

# Option d
True Negatives

# OUTPUT FORMAT: json

{
  "properties": {
    "option_a_explanation": {
      "description": "Detailed explanation of why the option 'a' is correct or not.",
      "title": "Option A Explanation",
      "type": "string"
    },
    "option_b_explanation": {
      "description": "Detailed explanation of why the option 'b' is correct or not.",
      "title": 

In [17]:
response = client.chat.completions.create(
    # model="llama3-8b-8192",
    model="llama-3.1-70b-versatile",
    messages=[{
        "role": "user",
        "content": prompt_revision_fromatted
    }],
    temperature=0.5,
    # max_tokens=2420,
    top_p=0.95,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

completion = response.choices[0].message
llama_analysis = json_repair.loads(completion.content)
llama_analysis

{'option_a_explanation': 'This option is correct because True Positives represent the cases that are correctly classified as positive. In the context of machine learning, this term is used to describe the number of instances that are correctly predicted as belonging to the positive class.',
 'option_b_explanation': 'This option is incorrect because False Positives represent the cases that are incorrectly classified as positive. In other words, these are the instances that do not belong to the positive class but are predicted as such.',
 'option_c_explanation': 'This option is incorrect because False Negatives represent the cases that are incorrectly classified as negative. In other words, these are the instances that belong to the positive class but are predicted as not belonging to it.',
 'option_d_explanation': 'This option is incorrect because True Negatives represent the cases that are correctly classified as negative. In other words, these are the instances that do not belong to t

## OpenAI

In [19]:
gemeni_analysis["correct_answer"] == llama_analysis['correct_answer']

True

In [72]:
from openai import OpenAI

client = OpenAI()

In [ ]:
prompt_merge = (
    "# BACKGROUND INFORMATION OF YOUR ROLE:\n"
    "You are an expert in machine learning.\n"
    "You working (with some other professionals) verifying if a question from a bank of question is ok to be used in a job interview.\n"
    "The question has already been analized by other professionals. Their analysis is below.\n"
    "# YOUR TASK\n"
    "your task is to give a detailed explanation for each option taken in consideration the analysis from the experts.\n"
    "You should also take in consideration the choosen"
)

In [77]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": prompt_revision_fromatted},
        # {"role": "user", "content": "hello"}
    ],
    response_format=ExplanationSchema,
)

In [ ]:
completion.choices[0].message.parsed